### Debug projection step
Key takeaway information: 
1. use **sparse** storage (**csr_matrix** is better than **csc_matrix**, since csr format is more suitable for matrix-vector product) (*cf*. https://docs.scipy.org/doc/scipy/reference/sparse.html);
2. when there is sparsity, use **lsqr** or spsolve, etc. to utilize the sparsity;
3. when no sparsity, cache **Cholesky factorization** or still use **lsqr**.

In [1]:
import numpy as np
import time
import scipy.sparse.linalg
import scipy.sparse

In [2]:
# generate consensus matrix
m0 = 100
n0 = 1000
N = 10   # Number of splits
I = np.eye(n0)
Z = np.zeros((n0,n0))
A_consensus_list = []
for i in range(N):
    Ai = []
    for j in range(N+1):
        if j == i:
            Ai.append(I)
        elif j == i+1:
            Ai.append(-I)
        else:
            Ai.append(Z)
    A_consensus_list.append(Ai)
A_consensus = np.block(A_consensus_list)

In [7]:
# compute AAT
A_consensus_sp = scipy.sparse.csr_matrix(A_consensus)
t1 = time.time()
AAT = A_consensus.dot(A_consensus.T)
t2 = time.time()
print('dense storage of A, AAT computation = {} seconds'.format(t2 - t1))
t1 = time.time()
AAT = A_consensus_sp.dot(A_consensus_sp.T)
t2 = time.time()
print('sparse storage of A, AAT computation = {} seconds'.format(t2 - t1))
print('AAT # nonzeros = {}'.format(AAT.count_nonzero()))

dense storage of A, AAT computation = 19.78760600090027 seconds
sparse storage of A, AAT computation = 0.06254887580871582 seconds
AAT # nonzeros = 28000


In [8]:
# generate random x and b = 0
x = np.random.randn(n0*(N+1))
b = np.zeros(n0*N)
Ax_b = A_consensus_sp.dot(x) - b

In [20]:
# solve the projection step least squares
# atol and btol controls the final accuracy of the solution
t1 = time.time()
res = scipy.sparse.linalg.lsqr(AAT,Ax_b,atol=1e-16,btol=1e-16)
t2 = time.time()
print('sparse storage of A, projection step using lsqr = {} seconds'.format(t2 - t1))

sparse storage of A, projection step using lsqr = 0.008413076400756836 seconds


In [21]:
# final output accuracy test
acc = np.linalg.norm(AAT.T.dot(AAT).dot(res[0]) - AAT.T.dot(Ax_b))
print('Final output accuracy in terms of the normal equation = {}'.format(acc))

Final output accuracy in terms of the normal equation = 2.346049526259128e-13


#### Some other alternative approaches
1. abstract **linear operator** input to lsqr (should be more efficient when A is sparse but AAT is not)
2. spsolve (only guaranteed to work when AAT is full rank)

In [33]:
# use abstract linear operator input to fully utilize the sparsity in A
def Ax(x):
    """Returns A*x"""
    return A_consensus_sp.dot(A_consensus_sp.T.dot(x))

def Atb(b):
    """Returns A'*b"""
    return A_consensus_sp.dot(A_consensus_sp.T.dot(b))

AAT_op = scipy.sparse.linalg.LinearOperator((n0*N,n0*N), matvec=Ax, rmatvec=Atb)

t1 = time.time()
res_op = scipy.sparse.linalg.lsqr(AAT_op,Ax_b,atol=1e-16,btol=1e-16)
t2 = time.time()
print('sparse storage of A, linear operator version of AAT, projection step using lsqr = {} seconds'.format(t2 - t1))

sparse storage of A, linear operator version of AAT, projection step using lsqr = 0.013865947723388672 seconds


In [37]:
# another method: solve using spsolve (assuming that AAT is full rank)
# but if AAT is not full rank, then this method may fail
t1 = time.time()
res_tmp = scipy.sparse.linalg.spsolve(AAT,Ax_b)
t2 = time.time()
print('sparse storage of A, projection step using spsolve = {} seconds'.format(t2-t1))
print('Final output accuracy = {}'.format(np.linalg.norm(AAT.dot(res_tmp)-Ax_b)))

sparse storage of A, projection step using spsolve = 0.008343219757080078 seconds
Final output accuracy = 2.1480537460527604e-14


In [38]:
# check formats of the sparse matrices
print('A_consensus_sp is {}'.format(A_consensus_sp.getformat()))
print('A_consensus_sp.T is {}'.format(A_consensus_sp.T.getformat()))
print('AAT is {}'.format(AAT.getformat()))

A_consensus_sp is csr
A_consensus_sp.T is csc
AAT is csr


In [39]:
# A_consensus_T_sp = scipy.sparse.csr_matrix(A_consensus_sp.T)
# don't do this, multiply two csr_matrix are terribly slow
# AAT.getformat()

#print(np.linalg.norm(AAT.dot(res[0])-Ax_b))

# res2 = np.linalg.solve(AAT,Ax_b)

# t1 = time.time()
# res = np.linalg.lstsq(AAT,Ax_b)
# t2 = time.time()
# t2 - t1